# Spreadsheet for English Lessons

using Selenium to get data from https://www.englishgrammar.org/lessons/

In [2]:
import numpy as np
import pandas as pd
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

#### access website

In [3]:
browser = webdriver.Chrome('/Users/liannelelis/Downloads/chromedriver')
browser.get('https://www.englishgrammar.org/lessons/')
browser.maximize_window()

/var/folders/x7/g3jn_87d3cldljmb204fyzwm0000gn/T/ipykernel_5329/3739795200.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('/Users/liannelelis/Downloads/chromedriver')


#### load page until end of page

In [9]:
loadMore = browser.find_element(By.PARTIAL_LINK_TEXT, 'Load More')
loadMore.click()

##### infinite scrolling

In [11]:
from time import sleep
# Set sleep time for the page to load on scroll
SCROLL_PAUSE_TIME = 2

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

count = 0

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    count += 1

print(f"{count} scrolls")
print("End of page reached")

sleep(2)

81 scrolls
End of page reached


#### data webscraping

In [14]:
date, lessons, links, pdf = ([] for i in range(4)) 

parent = browser.find_element(By.ID, 'pnlInfScroll_PostsList')
a = parent.find_elements(By.TAG_NAME,'a')
li = parent.find_elements(By.TAG_NAME,'li')

for lnk in a:
    l = lnk.get_attribute('href')
    if 'pdf' in l:
        pdf.append(l)
    else:
        links.append(l)

for elements in li:
    lessons.append(elements.text[10:])
    date.append(elements.text[0:8])

In [16]:
browser.close()

#### dataframe

In [61]:
english_lessons = pd.DataFrame({'Date': date,
                               'Lessons': lessons,
                               'URL': links,
                               'PDF': pdf})

In [63]:
# converting the string to datetime format
english_lessons['Date'] = pd.to_datetime(english_lessons['Date'], format='%y/%m/%d')

# clickable link in  dataFrame
from IPython.display import HTML,display
#display(HTML(english_lessons.to_html(render_links=True, escape=False)))
display(HTML(english_lessons[0:5].to_html(render_links=True, escape=False) ) )

,Date,Lessons,URL,PDF
0,2023-04-23,Gap Filling Vocabulary Exercise,https://www.englishgrammar.org/gap-filling-vocabulary-exercise-8/,https://www.englishgrammar.org/gap-filling-vocabulary-exercise-8/?pdf=16176
1,2023-04-23,Intermediate Level Gap Filling Vocabulary Exercise,https://www.englishgrammar.org/intermediate-level-gap-filling-vocabulary-exercise/,https://www.englishgrammar.org/intermediate-level-gap-filling-vocabulary-exercise/?pdf=16170
2,2022-04-23,Prepositions Quiz,https://www.englishgrammar.org/prepositions-quiz-31/,https://www.englishgrammar.org/prepositions-quiz-31/?pdf=16152
3,2021-04-23,Prepositions Quiz,https://www.englishgrammar.org/prepositions-quiz-30/,https://www.englishgrammar.org/prepositions-quiz-30/?pdf=16149
4,2021-04-23,Gap Filling Vocabulary Exercise,https://www.englishgrammar.org/gap-filling-vocabulary-exercise-5/,https://www.englishgrammar.org/gap-filling-vocabulary-exercise-5/?pdf=16146


### to do
- group the lessons by NLP
- build a streamlit app